In [ ]:
%pip install einops
%pip install peft
%pip install trl
%pip install tensorboard
%pip install -U transformers
%pip install -U accelerate datasets 
%pip install -q https://github.com/jllllll/bitsandbytes-windows-webui/releases/download/wheels/bitsandbytes-0.41.1-py3-none-win_amd64.whl
%pip install tokenizers==0.15.0
%pip install torch==2.1.2+cu121 --index-url https://download.pytorch.org/whl/cu121
%pip install torchaudio==2.1.2+cu121 --index-url https://download.pytorch.org/whl/cu121
%pip install torchvision==0.16.2+cu121 --index-url https://download.pytorch.org/whl/cu121
%pip install transformers==4.35.2
%pip install ipywidgets


In [168]:
import os
from dataclasses import dataclass, field
from typing import Optional
import json

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    Trainer
)
from tqdm.notebook import tqdm

from trl import SFTTrainer

In [179]:
dataset = load_dataset("json", data_files="./2011_2023_event_des.json", field='data', split='all')
dataset = dataset.train_test_split(test_size=0.1)


Found cached dataset json (C:/Users/danm/.cache/huggingface/datasets/json/default-75cc6199393bf5a9/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


In [170]:
dataset

DatasetDict({
    train: Dataset({
        features: ['train'],
        num_rows: 717166
    })
    test: Dataset({
        features: ['train'],
        num_rows: 79686
    })
})

In [183]:
tokenizer = AutoTokenizer.from_pretrained("../models/phi-2-mlb/tokenizer_merged", trust_remote_code=True)
tokenizer.pad_token = "<PAD>"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [185]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False,
)

In [189]:
model = AutoModelForCausalLM.from_pretrained(
        "../models/phi-2", 
        quantization_config=bnb_config, 
        device_map = 'auto',
        trust_remote_code=True,
        use_auth_token=True,
    )
model.config.eos_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [190]:
model.resize_token_embeddings(len(tokenizer))

Embedding(60359, 2560)

In [191]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True) 

lora_config = LoraConfig(
    r=32, 
    lora_alpha=32, 
    target_modules = [ "q_proj", "k_proj", "v_proj", "dense" ],
    modules_to_save = ["lm_head", "embed_tokens"],
    lora_dropout=0.1, 
    bias="none", 
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

model.config.use_cache = False

In [192]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'k_proj', 'v_proj','dense','fc1','fc2',]
)

In [193]:
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [194]:
# dataset-specific parameters
bs=8    # batch size for training
bs_eval=16    # batch size for evaluation
ga_steps=16  # gradient accumulation steps
lr=0.00002  # learning rate
epochs=1

steps_per_epoch=len(dataset["train"])//(bs*ga_steps)

args = TrainingArguments(
    output_dir="../models/phi-2-mlb",
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs_eval,
    evaluation_strategy="steps",
    logging_steps=1,
    eval_steps=steps_per_epoch//2,    # 2 evals per epoch
    save_steps=steps_per_epoch//100,
    save_total_limit=3,     # save once per epoch
    gradient_accumulation_steps=ga_steps,
    num_train_epochs=epochs,
    lr_scheduler_type="constant",
    optim="paged_adamw_32bit",      # val_loss will go nan with paged_adamw_8bit
    learning_rate=lr,
    group_by_length=False,
    bf16=True,        
    ddp_find_unused_parameters=False,
    
)



In [195]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    peft_config=lora_config,
    dataset_text_field="train",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=args,
    packing=False,
)

Map:   0%|          | 0/717166 [00:00<?, ? examples/s]

Map:   0%|          | 0/79686 [00:00<?, ? examples/s]

In [ ]:
trainer.train(resume_from_checkpoint="../models/phi-2-mlb/checkpoint-1344")

In [196]:
trainer.train()

  0%|          | 0/5602 [00:00<?, ?it/s]

{'loss': 5.8374, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.8606, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.7135, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.6986, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.6031, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.6055, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.5103, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.4549, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.4295, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.3531, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.3036, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.2769, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.1592, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.142, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.1154, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 4.9963, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 5.0159, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 4.9626, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 4.

c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-packages\peft\utils\save_and_load.py:148: UserWarning: Could not find a config file in ../models/phi-2 - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 2.9528, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.8366, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.7526, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.8025, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.7565, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.684, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.6601, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.5837, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.5416, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.4728, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.4205, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.4418, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.3281, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.3345, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.2849, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.2055, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.2384, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 2.1677, 'learning_rate': 2e-05, 'epoch':

c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-packages\peft\utils\save_and_load.py:148: UserWarning: Could not find a config file in ../models/phi-2 - will assume that the vocabulary was not modified.
  warnings.warn(


...

c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-packages\peft\utils\save_and_load.py:148: UserWarning: Could not find a config file in ../models/phi-2 - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 0.3286, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3311, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3184, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3121, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.32, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3103, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3122, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3156, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3144, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3187, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3172, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3113, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3179, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3073, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.309, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3206, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3107, 'learning_rate': 2e-05, 'epoch': 0.49}
{'loss': 0.3159, 'learning_rate': 2e-05, 'epoch': 0

c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-packages\peft\utils\save_and_load.py:148: UserWarning: Could not find a config file in ../models/phi-2 - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 0.3156, 'learning_rate': 2e-05, 'epoch': 0.5}


  0%|          | 0/4981 [00:00<?, ?it/s]

In [ ]:
trainer.save_model("../models/phi-2-mlb/")

In [ ]:
# 

In [148]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel
import torch

# base model
base_path="../models/phi-2"  


# adapters: path to folder with adapter_model.safetensors
adapter_path="../models/phi-2-mlb/" 

# # where to save merged model
save_to="../models/phi-2-mlb/"       

generation_config = GenerationConfig(
    max_new_tokens=100, 
    temperature=0.7,
    top_p=0.1,
    top_k=40,
    repetition_penalty=1.18,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

# Load LoRA and merge
merged_model = PeftModel.from_pretrained(model, adapter_path)
merged_model = merged_model.merge_and_unload()

merged_model.save_pretrained(save_to, safe_serialization=True, max_shard_size='4GB')
tokenizer.save_pretrained(save_to)
generation_config.save_pretrained(save_to)

c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-packages\peft\tuners\lora\bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel
import torch

model_path="../models/phi-2-mlb/"   

model = AutoModelForCausalLM.from_pretrained(
        model_path, 
        quantization_config=bnb_config, 
        device_map = 'auto',
        trust_remote_code=True,
        use_auth_token=True,
    )

model.config.eos_token_id = tokenizer.eos_token_id

prompt= "Instruct: what is the outcome of {\"input\": {\"pitcher\": {\"id\": 573204, \"name\": \"caleb thielbar\"}, \"batter\": {\"id\": 488726, \"name\": \"michael brantley\"}}}? \n"

input_tokens = tokenizer(prompt, return_tensors="pt").to("cuda")
output_tokens = model.generate(**input_tokens, max_new_tokens=128)

output = tokenizer.decode(
    output_tokens[0][len(input_tokens[0]):],
    skip_special_tokens=True
    )               

print(output)